# 基于Mindspore构造全连接层

全连接层指的是层中的每个节点都会连接它下一层的所有节点，它是模仿人脑神经结构来构建的。用来把前边提取到的特征综合起来。由于其全相连的特性，一般全连接层的参数也是最多的。全连接层则起到将学到的“分布式特征表示”映射到样本标记空间的作用。

公式如下：
$$output=activation(X*kernel+bias)$$

## 使用示例

In [ ]:
class mindspore.nn.Dense(in_channels, out_channels, weight_init='normal', bias_init='zeros', has_bias=True, activation=None)

In [1]:
from mindspore.common.tensor import Tensor
import mindspore.nn as nn
import numpy as np
import mindspore.common.dtype as mstype

# 构建一个2*3的矩阵，进行Dense运算
x = Tensor(np.array([[180, 234, 154], [244, 48, 247]]), mstype.float32)
net = nn.Dense(3, 4)
# 输出为2*4矩阵
output = net(x)
print(output)
print(output.shape)



[[-1.1079562  -0.75799847 -7.678297    1.2090455 ]
 [-4.5353317   0.03348327 -4.456973    1.5525969 ]]
(2, 4)


### 参数说明
``` python
in_channels (int) - Dense层输入Tensor的空间维度。

out_channels (int) - Dense层输出Tensor的空间维度。

weight_init (Union[Tensor, str, Initializer, numbers.Number]) - 权重参数的初始化方法。数据类型与 x 相同。str的值引用自函数 initializer。默认值：'normal'。

bias_init (Union[Tensor, str, Initializer, numbers.Number]) - 偏置参数的初始化方法。数据类型与 x 相同。str的值引用自函数 initializer。默认值：'zeros'。

has_bias (bool) - 是否使用偏置向量 bias 。默认值：True。

activation (Union[str, Cell, Primitive, None]) - 应用于全连接层输出的激活函数。可指定激活函数名，如’relu’，或具体激活函数，如mindspore.nn.ReLU()。默认值：'None'。
```

# 自行实现Mindspore Dense API

## 导入所需包

In [2]:
from mindspore.common.tensor import Tensor
from mindspore.common.initializer import initializer
from mindspore.ops import operations as P
from mindspore.common.parameter import Parameter
from mindspore._extends import cell_attr_register
from mindspore._checkparam import Rel, Validator
from mindspore.nn.cell import Cell
from mindspore.nn.layer.activation import get_activation
import mindspore.common.dtype as mstype

## 具体实现

In [3]:


class Dense(Cell):
    @cell_attr_register(attrs=['has_bias', 'activation'])
    def __init__(self,
                 in_channels,
                 out_channels,
                 weight_init='normal',
                 bias_init='zeros',
                 has_bias=True,
                 activation=None):
        """Initialize Dense."""
        super(Dense, self).__init__()
        self.in_channels = Validator.check_positive_int(in_channels, "in_channels", self.cls_name)
        self.out_channels = Validator.check_positive_int(out_channels, "out_channels", self.cls_name)
        self.has_bias = Validator.check_bool(has_bias, "has_bias", self.cls_name)
        self.reshape = P.Reshape()
        self.shape_op = P.Shape()
        self.weight = Parameter(initializer(weight_init, [out_channels, in_channels]), name="weight")

        self.bias = None
        if self.has_bias:
            if isinstance(bias_init, Tensor):
                if bias_init.ndim != 1 or bias_init.shape[0] != out_channels:
                    raise ValueError(f"For '{self.cls_name}', bias init shape error. The ndim of 'bias_init' must "
                                     f"be equal to 1, and the first dim must be equal to 'out_channels'. But got "
                                     f"'bias_init': {bias_init}, 'out_channels': {out_channels}.")
            self.bias = Parameter(initializer(bias_init, [out_channels]), name="bias")
            self.bias_add = P.BiasAdd()

        self.matmul = P.MatMul(transpose_b=True)
        self.activation = get_activation(activation) if isinstance(activation, str) else activation
        self.activation_flag = self.activation is not None

    def construct(self, x):
        x_shape = self.shape_op(x)
        if len(x_shape) != 2:
            x = self.reshape(x, (-1, x_shape[-1]))
        x = self.matmul(x, self.weight)
        if self.has_bias:
            x = self.bias_add(x, self.bias)
        if self.activation_flag:
            x = self.activation(x)
        if len(x_shape) != 2:
            out_shape = x_shape[:-1] + (-1,)
            x = self.reshape(x, out_shape)
        return x


# 使用示例

In [4]:
# 构建一个2*3的矩阵，进行Dense运算
x = Tensor(np.array([[180, 234, 154], [244, 48, 247]]), mstype.float32)
net = Dense(3, 4)
# 输出为2*4矩阵
output = net(x)
print(output)
print(output.shape)

[[-1.9869218  -1.7707653   2.3805907   0.07664371]
 [-0.3781587  -0.6915364   1.6991204  -2.827205  ]]
(2, 4)
